##**Training a YOLO11n Model for Inference on Android and Integration with Unity (Quest 3)**

### **Step 1 : Install Required Dependencies**
Run the following command to install and update all necessary packages for training and exporting the YOLO11n model

In [ ]:
!pip install --upgrade --force-reinstall ultralytics onnxruntime-gpu onnxruntime onnx opencv-python matplotlib

After installation, **Google Colab** may prompt you to restart the runtime.
Please click “**Restart session**” before proceeding.

**After Restarting the Runtime**

Once you click “**Restart session**” (as required in Step 1), please return to this notebook and continue directly from Step 2 to download your dataset and proceed with training.

### **Step 2 : Download Dataset from Roboflow**
Run the following code to install Roboflow, authenticate using your API key, and download the specified dataset in YOLOv8 format.

This step also handles manual extraction if the roboflow.zip file already exists.

In [ ]:
!pip install roboflow
import zipfile, os
from roboflow import Roboflow

api_key = ""
workspace_name = ""
project_name = ""
version_number = 2
format_type = ""

rf = Roboflow(api_key=api_key)
project = rf.workspace(workspace_name).project(project_name)
dataset = project.version(version_number).download(format_type)


zip_path = "roboflow.zip"
if os.path.exists(zip_path):
    print("Found roboflow.zip — extracting manually...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall("roboflow_dataset")
    dataset_dir = "/content/roboflow_dataset"
else:
    dataset_dir = dataset.location

print("Dataset directory:", dataset_dir)
!ls -R $dataset_dir


data_yaml = f"{dataset.location}/data.yaml"
print("Using data.yaml:", data_yaml)

### **Step 3 : Train the YOLO11n Model**
Run the code below to initialize the YOLO11n model and begin training using your Roboflow dataset.

This configuration includes augmentation settings optimized for lightweight Android inference.

In [ ]:
from ultralytics import YOLO
model = YOLO("yolo11n.pt")
model.train(
    data=data_yaml,
    epochs=100,
    imgsz=320,
    batch=16,
    device=0,
    workers=2,
    patience=10,

    augment=True,
    degrees=5,
    translate=0.05,
    scale=0.10,
    shear=0.0,
    perspective=0.001,

    flipud=0.0,
    fliplr=0.5,

    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,

    mosaic=0.0,
    mixup=0.0,
)

### **Step 4 : Load the Trained Best Weights**
After training completes, load the best-performing model weights generated by YOLO.

These weights will be used for exporting to ONNX in the following step.

In [ ]:
model = YOLO("runs/detect/train/weights/best.pt")

### **Step 5 : Export the Model to ONNX Format**
Export the trained YOLO11n model to ONNX format, which is required for running inference on Android through JNI.

In [ ]:
model.export(
    format="onnx",
    imgsz=320,
    dynamic=False,
    simplify=False,
    opset=12
)

### **Step 6 : Download the ONNX Model**
Download the exported ONNX file to your local machine.

This file will later be integrated into your Android project for JNI-based inference.

In [ ]:
from google.colab import files
files.download('runs/detect/train/weights/best.onnx')